In [1]:
import pandas as pd
import jsonlines
import glob
from tqdm.auto import tqdm

In [2]:
fnames = glob.glob('../data_large/pantip-large/*.jsonl')
len(fnames)

72

In [3]:
# %%time

# content_dfs = []
# comment_dfs = []
# for fname in tqdm(fnames):
#     with jsonlines.open(fname) as reader:
#         for obj in tqdm(reader):
#             if 'comment' in obj.keys():
#                 d = {
#                     'datasource_url': obj['datasource_url'],
#                     'content_id': obj['content_id'],
#                     'comment_id': obj['comment_id'],
#                     'order_comment': obj['order_comment'],
#                     'type':obj['type'],
#                     'comment': obj['comment'],
#                     'like_score': obj['like_score'],
#                     'feel_heart': obj['feel_heart'],
#                     'feel_laugh': obj['feel_laugh'],
#                     'feel_love': obj['feel_love'],
#                     'feel_sad': obj['feel_sad'],
#                     'feel_horror': obj['feel_horror'],
#                     'feel_wow': obj['feel_wow'],
#                 }
#                 comment_dfs.append(d)
#             elif ('content' in obj.keys()) and (obj['total_field'] > 0):
#                 d = {
#                     'datasource_url': obj['datasource_url'],
#                     'content_id': obj['content_id'],
#                     'type':obj['type'],
#                     'title': obj['title'],
#                     'body': obj['content'],
#                     'total_field': obj['total_field'],
#                     'tags': obj['tags'],
#                     'like_score': obj['like_score'],
#                     'feel_heart': obj['feel_heart'],
#                     'feel_laugh': obj['feel_laugh'],
#                     'feel_love': obj['feel_love'],
#                     'feel_sad': obj['feel_sad'],
#                     'feel_horror': obj['feel_horror'],
#                     'feel_wow': obj['feel_wow'],
#                 }
#                 content_dfs.append(d)    

# comment_df = pd.DataFrame(comment_dfs)
# content_df = pd.DataFrame(content_dfs)

In [4]:
# content_df = content_df.drop_duplicates()
# #remove threads with different snapshots
# content_df['rnk'] = content_df.sort_values(['total_field',],ascending=[False]) \
#              .groupby(['content_id']) \
#              .cumcount() + 1
# content_df = content_df[content_df.rnk==1]
# content_df.to_csv('../data_large/content_df.csv', index=None)


# comment_df = comment_df.drop_duplicates()
# comment_df['rnk'] = comment_df.groupby(['content_id','comment_id']).cumcount() + 1
# comment_df = comment_df[comment_df.rnk==1]
# comment_df.to_csv('../data_large/comment_df.csv', index=None)


# comment_df = pd.read_csv('../data_large/comment_df.csv')
# content_df = pd.read_csv('../data_large/content_df.csv')

In [5]:
# #check if content deduplicated successfully
# content_df.shape, content_df.content_id.nunique(), content_df.drop_duplicates().shape

In [6]:
# #check if content deduplicated successfully
# comment_df.shape, comment_df.comment_id.nunique(), comment_df.drop_duplicates().shape

In [43]:
# all_df = content_df.merge(comment_df, on='content_id')
# all_df = all_df.drop(['rnk_x','rnk_y','datasource_url_y'],1)
#filter out later sub-comments
# all_df = all_df[all_df.order_comment.map(lambda x: '-' not in str(x))]
#create interact count
# all_df['nb_interact'] = all_df[['like_score_y',
#        'feel_heart_y', 'feel_laugh_y', 'feel_love_y', 'feel_sad_y',
#        'feel_horror_y', 'feel_wow_y']].sum(1)

#only keep top 5 comments with highest interacts
# all_df['rnk'] = all_df.sort_values(['nb_interact','like_score_y','feel_heart_y'], \
#              ascending=False) \
#              .groupby(['content_id']) \
#              .cumcount() + 1
# all_df = all_df[all_df.rnk<=5].drop('rnk',1)
# all_df = all_df.dropna()
# all_df.to_csv('../data_large/all_df.csv', index=None)
all_df.shape, all_df.content_id.nunique()

((11403910, 26), 3162840)

In [38]:
all_df = pd.read_csv('../data_large/all_df.csv')
all_df.head(20)

,datasource_url_x,content_id,type_x,title,body,total_field,tags,like_score_x,feel_heart_x,feel_laugh_x,...,type_y,comment,like_score_y,feel_heart_y,feel_laugh_y,feel_love_y,feel_sad_y,feel_horror_y,feel_wow_y,nb_interact
0,https://pantip.com/topic/36188003,36188003,content,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินก...,4,"Social Network,Twitter",0,0,0,...,comment,ส่งหาใครคะ ดีเอ็มต้องโฟโลทั้งสองฝ่ายปะอะ,0,0,0,0,0,0,0,0
1,https://pantip.com/topic/36188003,36188003,content,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินก...,4,"Social Network,Twitter",0,0,0,...,comment,ไม่ต้อง follow กันและกัน ก็ส่งได้นะส่งมาแล้วแต...,0,0,0,0,0,0,0,0
2,https://pantip.com/topic/36188003,36188003,content,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินก...,4,"Social Network,Twitter",0,0,0,...,comment,จขกท.แก้ได้ยังค่ะ เราก็เปน ฝากด้วยนะคะ(แต่เราส...,0,0,0,0,0,0,0,0
3,https://pantip.com/topic/36188003,36188003,content,ส่งDMในทวิตไม่ได้ ทำยังไงดี?,DMในทวิตไม่ได้ค่ะ พอส่งมันก็ขึ้นว่า การดำเนินก...,4,"Social Network,Twitter",0,0,0,...,comment,แก้ได้ยังคะ?เราก็เป็น,0,0,0,0,0,0,0,0
4,https://pantip.com/topic/36188206,36188206,content,ใช้เน็ตบ้านทรูขึ้น ERR_CONNECTION_RESET บ่อยม...,ใช้เน็ตบ้านทรูขึ้น ERR_CONNECTION_RESET บ่อยม...,3,"อินเทอร์เน็ต,ADSL,Network,true online,Wi-Fi",0,0,0,...,comment,ไม่ไหว เบื่อ ที่เรียกซ่อม,0,0,0,0,0,0,0,0
5,https://pantip.com/topic/36188206,36188206,content,ใช้เน็ตบ้านทรูขึ้น ERR_CONNECTION_RESET บ่อยม...,ใช้เน็ตบ้านทรูขึ้น ERR_CONNECTION_RESET บ่อยม...,3,"อินเทอร์เน็ต,ADSL,Network,true online,Wi-Fi",0,0,0,...,comment,ดูเรื่องการตั้งเวลาในเครื่อง,0,0,0,0,0,0,0,0
6,https://pantip.com/topic/36188206,36188206,content,ใช้เน็ตบ้านทรูขึ้น ERR_CONNECTION_RESET บ่อยม...,ใช้เน็ตบ้านทรูขึ้น ERR_CONNECTION_RESET บ่อยม...,3,"อินเทอร์เน็ต,ADSL,Network,true online,Wi-Fi",0,0,0,...,comment,สวัสดีครับคุณสมาชิกหมายเลข 3282595ผมขอข้อมูลใน...,0,0,0,0,0,0,0,0
7,https://pantip.com/topic/36188128,36188128,content,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับม...,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับม...,3,"Freeware,ซอฟต์แวร์,Multimedia Software,ตัดต่อว...",0,0,0,...,comment,Windows Movie Maker,0,3,0,0,0,0,0,3
8,https://pantip.com/topic/36188128,36188128,content,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับม...,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับม...,3,"Freeware,ซอฟต์แวร์,Multimedia Software,ตัดต่อว...",0,0,0,...,comment,Sony vegas 13.0 //สำหรับผมเเล้วใช้ได้หลากหลาย ...,0,0,0,0,0,0,0,0
9,https://pantip.com/topic/36188128,36188128,content,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับม...,จะตัดต่อ วิดีโอ ใช้โปรแกรมอะไรทำดีครับ สำหรับม...,3,"Freeware,ซอฟต์แวร์,Multimedia Software,ตัดต่อว...",0,0,0,...,comment,เอาง่าย ไม่แฮงค์ ไม่หงุดหงิดเรนเดอร์ไวแอฟทันสม...,0,0,0,0,0,0,0,0


In [39]:
all_df[all_df.title.map(lambda x: '?' in str(x))].sort_values('nb_interact',ascending=False).head(20)

,datasource_url_x,content_id,type_x,title,body,total_field,tags,like_score_x,feel_heart_x,feel_laugh_x,...,type_y,comment,like_score_y,feel_heart_y,feel_laugh_y,feel_love_y,feel_sad_y,feel_horror_y,feel_wow_y,nb_interact
5497454,https://pantip.com/topic/31812211,31812211,content,"วง ""ดีทูบี"" (D2B) นี่สมัยก่อนดังขนาดไหน ?????...",พอดีเราไม่ค่อยชอบฟังเพลงเท่าไหร่ แต่เกิดทันค่ะ???,564,"นักร้องนักดนตรี,นักร้องนักดนตรีชาย,เพลง",336,119,5,...,comment,ไม่สามารถบรรยายออกมาเป็นคำพูดได้ค่ะ ว่าดังแค่ไ...,1467,666,9,49,70,6,473,2740
8990083,https://pantip.com/topic/33102515,33102515,content,คุณรู้สึกยังไงกับการ ที่ บอย ปกรณ์ และ ครอบครั...,ทาง ig เขาก็มีกฎ ตั้งแต่แรกแล้ว ห้ามเด็กอายุเท...,341,"ปกรณ์ ฉัตรบริรักษ์ (บอย),นักแสดงชาย,Instagram",627,180,8,...,comment,ครอบครัวนี้............คนโต เป็นดารา ....วัน...,1361,693,182,0,3,26,16,2281
8906174,https://pantip.com/topic/33649724,33649724,content,"ถึงเวลาแล้วหรือยัง ที่จะเลิกให้ราคากับเพจ ""อีเ...",ตอนแรกก็ติดตามนะ ใช้คำพูดมีสีสันดี อ่านแล้วได้...,400,นักแสดง,399,163,97,...,comment,จริงๆ คนเล่นเพจอิเจี๊ยบก็พอๆกับคนเล่นพันทิปนี่...,1214,761,89,2,1,1,2,2070
5442529,https://pantip.com/topic/32257049,32257049,content,เพลงอะไรมั้ง ที่เคยคิดว่าร้องแบบนี้มาตลอด จนได...,เคยมั้ยที่ฟังเพลงไหน แล้วคิดว่ามันร้องแบบนี้มา...,874,"เพลงไทยสากล,เพลง,ดนตรี,การร้อง,นักร้องนักดนตรี",315,40,420,...,comment,"ที่จำฝังใจ ตอนเด็กๆ "" สี่กุมาร หาญกล้า ตีลังกา...",546,137,904,2,1,0,1,1591
2437581,https://pantip.com/topic/30879413,30879413,content,"เมื่อจิ๋มฉันดำและขนดก, แล้วไง?",Pipob UdomittipongHoni Soit เป็นนิตยสารเก่าแก่...,201,"สุขภาพกาย,ความงาม",552,0,9,...,comment,แค่รักษาหน้ากับหุ่นให้ดูดีชั้นก็ลำบากจะแย่แล้ว...,606,218,606,7,5,3,0,1445
5442528,https://pantip.com/topic/32257049,32257049,content,เพลงอะไรมั้ง ที่เคยคิดว่าร้องแบบนี้มาตลอด จนได...,เคยมั้ยที่ฟังเพลงไหน แล้วคิดว่ามันร้องแบบนี้มา...,874,"เพลงไทยสากล,เพลง,ดนตรี,การร้อง,นักร้องนักดนตรี",315,40,420,...,comment,"เพลงนี้เลยฟังผิดตั้งแต่เด็กจาก""ใส่โองหินฝังดิน...",651,215,542,1,7,0,5,1421
8990082,https://pantip.com/topic/33102515,33102515,content,คุณรู้สึกยังไงกับการ ที่ บอย ปกรณ์ และ ครอบครั...,ทาง ig เขาก็มีกฎ ตั้งแต่แรกแล้ว ห้ามเด็กอายุเท...,341,"ปกรณ์ ฉัตรบริรักษ์ (บอย),นักแสดงชาย,Instagram",627,180,8,...,comment,คิดว่าคงต้องมองบอย ปกรณ์ใหม่แบบจริงจัง ไม่ใช่แ...,786,572,6,1,0,1,2,1368
9047414,https://pantip.com/topic/32687375,32687375,content,เจนี่ ทอป เรื่องนี้ต้องขยายมั้ย? มีอะไรในกอไผ่...,cr. เพจ อีเจี๊ยบ เลียบด่วนแก้ไขข้อความเมื่อ,450,"เจนี่ เทียนโพธิ์สุวรรณ,นักแสดง",413,72,1,...,comment,ถ้าครั้งนี้ทำอีก ...... รัฐต้องเข้ามามีบทบาทได...,523,139,614,2,0,3,3,1284
10038940,https://pantip.com/topic/33187027,33187027,content,เคยลืมอะไรแบบนี้รึเปล่า? มหาลัย 18+,สวัสดีครับ กระทู้นี้เป็นกระทู้แรกของผม ถ้าผิดพ...,567,"ภาคเหนือ,จังหวัดเชียงราย,ชีวิตวัยรุ่น",101,8,36,...,comment,สวัสดีค่ะเราเป็นคนที่ได้รับความเสียหายจากการที...,746,416,2,4,7,2,12,1189
5442531,https://pantip.com/topic/32257049,32257049,content,เพลงอะไรมั้ง ที่เคยคิดว่าร้องแบบนี้มาตลอด จนได...,เคยมั้ยที่ฟังเพลงไหน แล้วคิดว่ามันร้องแบบนี้มา...,874,"เพลงไทยสากล,เพลง,ดนตรี,การร้อง,นักร้องนักดนตรี",315,40,420,...,comment,Diamond ของ Rihanna ค่ะได้ยินเป็น เชนไดร์ท ไลค...,420,70,691,2,0,0,0,1183


In [51]:
all_df.iloc[74498,:][['title','body','comment']].tolist()

['พ่อแม่ไทยแท้ แต่ให้ลูกเรียกปะป๊า หม่าม๊า ใช่หรอ?',
 'เราเป็นคนนึงที่ยอมรับเลยว่า หงุดหงิดมาก เวลาเห็น พ่อแม่ไทยแท้ บ้างก็ตัวดำปี๋ทั้งพ่อทั้งแม่ ไม่มีเชื้อส่วนไหนเป็นคนจีน แต่ทำไมต้องให้ลูกเรียก ปะป๊า หม่าม๊า คือ เรียกว่า " พ่อ แม่ " มันไม่ดีกว่าหรอ คือแบบเคยให้ยินเด็กถามพ่อแม่ในที่สาธารณะว่า"ม๊าๆ ไมให้เติ้ลต้องเรียกม๊าว่าม๊า แล้วไมเพื่อนเรียกว่าแม่อ่ะ?"คือถ้าพ่อแม่เชื้อสายจีนอาจตอบไปว่า"อ๋อ .. เพราะเรามีเชื้อสายจีนลูก อย่างตอนป๊ากะม๊าเรียกอาม่า ก็ต้องเรียก ม๊าเหมือนกัน แปลว่าแม่ในภาษาไทยไงลูก"แต่ถ้าเป็นคนไทยแต่กำเนิด ใต้จ๋า เหนือจ๋า กลางจ๋า อีสานจ๋า พ่อแม่ตัวเองเรียก พ่อ แม่ อีป้อ อีแม่ ไรเทือกนี้แล้วจะไปตอบลูกยังไง?อ๋อ คือแบบว่า แม่อยากให้เก๋ๆอ่ะลูก ฟรุ้งฟริ้งๆ เห็นตอนเด็กๆ เพื่อนเรียกปะป๊า หม่าม๊า แม่เลยคิดว่าถ้ามีลูกจะให้เรียกแบบนั้น อย่างงั้นหรอคือเราคนไทยอ่ะค่ะ เราอาจคิดลบไปมั้ง แต่เราฟัง เห็น หรือเจอแล้วมันขัดหูขัดใจจริงๆนะคะเข้าใจว่าจะให้ลูกเรียกยังไงก็สิทธิ์ของครอบครัวเค้า เรียกอะไรก็ไม่ส่งผลกะการแสดงความเป็นพ่อ เป็นแม่คนหรอก แต่ก็เราคนไทย ใช้คำว่า พ่อ แม่ เราว่ามันคือคำที่มี

In [41]:
all_df = all_df.dropna()
all_df.shape

(11403910, 26)